# ReAct agente desde cero

In [ ]:
# Basado en https://til.simonwillison.net/llms/python-react-pattern

In [ ]:
import openai
import re
import os

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI()

In [ ]:
chat_completion = client.chat.completions.create(
    model = "gpt-4o-mini",
    # model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [ ]:
chat_completion.choices[0].message.content

In [ ]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [ ]:
prompt = """
Corres en un ciclo de Pensamiento, Acción, PAUSA, Observación.
Al final del ciclo, das una Respuesta.

Usa el Pensamiento para describir tus pensamientos sobre la pregunta que se te ha hecho.
Usa la Acción para realizar una de las acciones disponibles para ti, luego regresa a PAUSA.
La Observación será el resultado de ejecutar esas acciones.

Tus acciones disponibles son:

calcular:
Ejemplo: calcular: 4 * 7 / 3
Ejecuta un cálculo y devuelve el número, usa la sintaxis de punto flotante si es necesario.

peso_promedio_perro:
Ejemplo: peso_promedio_perro: Collie
Devuelve el peso promedio de un perro cuando se da la raza.

Ejemplo de sesión:

Pregunta: ¿Cuánto pesa un Bulldog?
Pensamiento: Debería buscar el peso del perro usando peso_promedio_perro.
Acción: peso_promedio_perro: Bulldog
PAUSA

Te llamarán nuevamente con esto:

Observación: Un Bulldog pesa 51 libras.

Luego respondes:

Respuesta: Un Bulldog pesa 51 libras.
""".strip()

In [ ]:
def calcular(what):
    return eval(what)

def peso_promedio_perro(name):
    if name in "Scottish Terrier": 
        return("El peso promedio de un Scottish Terrier es 20 libras")
    elif name in "Border Collie":
        return("El peso promedio de un Border Collie es 37 libras")
    elif name in "Toy Poodle":
        return("El peso promedio de un Toy Poodle es 7 libras")
    else:
        return("El peso promedio de un perro es 50 libras")



In [ ]:
abot = Agent(prompt)

In [ ]:
result = abot("Cuánto pesa un toy poodle?")
print(result)

In [ ]:
result = peso_promedio_perro("Toy Poodle")
print(result)

In [ ]:
next_prompt = "Observacion: {}".format(result)

In [ ]:
abot(next_prompt)

In [ ]:
for idx, message in enumerate(abot.messages):
    print(f"Mensaje {idx + 1}:")
    print(f"  Rol: {message['role']}")
    print(f"  Contenido: {message['content']}")
    print("-" * 40)

In [ ]:
abot = Agent(prompt)

In [ ]:
question = """Tengo dos perros, un border collie y un scottish terrier. \
Cuanto pesan entre los dos?"""
abot(question)

In [ ]:
next_prompt = "Observacion: {}".format(peso_promedio_perro("Border Collie"))
print(next_prompt)

In [ ]:
abot(next_prompt)

In [ ]:
next_prompt = "Observacion: {}".format(peso_promedio_perro("Scottish Terrier"))
print(next_prompt)

In [ ]:
abot(next_prompt)

In [ ]:
next_prompt = "Observacion: {}".format(eval("37 + 20"))
print(next_prompt)

In [ ]:
abot(next_prompt)

### Ahora automatizamos en una función

In [ ]:
action_re = re.compile('^Acción: (\w+): (.*)$')   # expresión regular para capturar secuencias de texto

acciones_disponibles = {
    "calcular": calcular,
    "peso_promedio_perro": peso_promedio_perro
}

In [ ]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        acciones = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if acciones:
            # There is an action to run
            accion, accion_input = acciones[0].groups()
            if accion not in acciones_disponibles:
                raise Exception("Acción desconocida: {}: {}".format(accion, accion_input))
            print(" -- corriendo {} {}".format(accion, accion_input))
            observacion = acciones_disponibles[accion](accion_input)
            print("Observacion:", observacion)
            next_prompt = "Observacion: {}".format(observacion)
        else:
            return

In [ ]:
question = """Tengo dos perros, un border collie y un scottish terrier. \
Cuanto pesan entre los dos?"""
query(question)